In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
%run -i ../../zkstats/core.py

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

# where(element > 30)
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
gt30_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 30].reshape(1,-1,1)
new_dummy_size = torch.tensor(gt30_dummy_data_tensor.size()[1] )
dummy_theory_output = torch.mean(gt30_dummy_data_tensor)



In [5]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.new_size = nn.Parameter(data = new_dummy_size,requires_grad = False )
        self.new_X = nn.Parameter(data = gt30_dummy_data_tensor,requires_grad = False )

    def forward(self,X):
        # where part
        # Many of these implementations are weird, but make it satisfy zkp of ezkl.
        len_ratio =  self.new_size/X.size()[1]
        result = torch.zeros(len_ratio*X.size()[1]).reshape(1,-1,1)
        index_array = torch.tensor(range(int(len_ratio*X.size()[1])))
        for index in index_array:
            result[0][index][0] = self.new_X[0][index][0]
        # constraint that new_X is indeed X where element is greater than 30
        check_bool = torch.sum((torch.abs(X[X>30].reshape(1,-1,1)-self.new_X)<0.01*self.new_X).double())

        if check_bool==self.new_size:
            # value from mean calculation
            value = torch.abs(torch.sum(result)-result.size()[1]*(self.w))<0.01*result.size()[1]*(self.w)
        else:
            value = torch.tensor(0)

        return (value, self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_43136/3656555878.py:14: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  index_array = torch.tensor(range(int(len_ratio*X.size()[1])))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_43136/3656555878.py:14: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  index_array = torch.tensor(range(int(len_ratio*X.size()[1])))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_43136/3656555878.py:15: TracerWarning: Iterating over a tensor might cause the 

check_bool:  tensor(236., dtype=torch.float64)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/utils.py:1686: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:ConstantOfShape, node name: /ConstantOfShape): input typestr: T1, has unsupported type: tensor(float) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/jit/serialization/export.cpp:1421.)
  _C._check_onnx_proto(proto)


In [11]:
# prover calculates settings, send to verifier
gt30_data_tensor = data_tensor[data_tensor > 30].reshape(1,-1,1)
new_size = torch.tensor(gt30_data_tensor.size()[1] )
print("new tensor: ", gt30_data_tensor.size())
# gt30_data_tensor = torch.reshape(gt30_data_tensor_raw, (1, int(gt30_data_tensor_raw.size()[0]), 1))
theory_output = torch.mean(gt30_data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.new_size = nn.Parameter(data = new_size,requires_grad = False )
        self.new_X = nn.Parameter(data = gt30_data_tensor,requires_grad = False )
    def forward(self,X):
        # where part
        # Many of these implementations are weird, but make it satisfy zkp of ezkl.
        len_ratio =  self.new_size/X.size()[1]
        result = torch.zeros(len_ratio*X.size()[1]).reshape(1,-1,1)
        index_array = torch.tensor(range(int(len_ratio*X.size()[1])))
        for index in index_array:
            result[0][index][0] = self.new_X[0][index][0]
        # constraint that new_X is indeed X where element is greater than 30
        check_bool = torch.sum((torch.abs(X[X>30].reshape(1,-1,1)-self.new_X)<0.01*self.new_X).double())

        if check_bool==self.new_size:
            # value from mean calculation
            value = torch.abs(torch.sum(result)-result.size()[1]*(self.w))<0.01*result.size()[1]*(self.w)
        else:
            value = torch.tensor(0)

        return (value, self.w)

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

new tensor:  torch.Size([1, 272, 1])
Theory_output:  tensor(52.3676)
check_bool:  tensor(272., dtype=torch.float64)
==== Generate & Calibrate Setting ====


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_43136/1416464578.py:25: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if check_bool==self.new_size:
thread '<unnamed>' panicked at src/graph/input.rs:493:42:
index out of bounds: the len is 0 but the index is 0
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: index out of bounds: the len is 0 but the index is 0

In [10]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 1.1819510459899902 seconds
Theory output:  tensor(52.3676)


spawning module 0


==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 52.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[17970410297053518904, 9046703063145816218, 2851759239208196922, 164045840560226117], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [1460628720732602093, 15005283833837523956, 127539993950163949, 306168462079750959]]], 'proof': '1267a75172347fa030369c8cbb5a79261c7dd7d733bfda2df9d2595c31544038101d5d6f369040ed90a8a83e1e974d01d28b3fbd918544773096dc6d3a71fc82244bd773fe908c1140aa24b317f71f2ef63fa46758a2ffc4c5c0f09fbc5c7cf8042cc8ba3d8fc8cf1f6b922b272b84d7b3cf52acc42a3d158da5562720b0a16e0f0cb526637827c73a75043a4ad33081c9f6016d86a7eff8258a5f8d6557d1682c92b9d10c59d629c341f8dea05f8963aa5003131f27d0c22226adf416c7f0ec205d15a01bebc8185c31489edac714c31f91bd89c535260f28ba61364c3ed6cc127a06c9ce3797ed1ea2ab0aadb475cf985644582d9a52570de28d93cfc002b9246fbd579aa81cf4b972ef10ecbb07cbcd5b62582b64787f5ce74f1ea2e11f6918f38dafddf9ad25c6a8a0b2c464b8d2d8a4a82b52674428fb6c30b31e803275095545835365fd8430d9e4cd8321c84a3e9baa165029f9ab238f253159943a7a00

In [9]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[17970410297053518904, 9046703063145816218, 2851759239208196922, 164045840560226117], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [1460628720732602093, 15005283833837523956, 127539993950163949, 306168462079750959]]]
proof boolean:  1.0
proof result 1 : 52.0
verified
